<a href="https://colab.research.google.com/github/bimal-bp/Predicting-sales-of-_Rossmann_store/blob/ML_MODELS/Predicting_sales_of_Rossmann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><u> Project Title : Sales Prediction : Predicting sales of a major store chain Rossmann</u></b>

## <b> Problem Description </b>

### Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

### You are provided with historical sales data for 1,115 Rossmann stores. The task is to forecast the "Sales" column for the test set. Note that some stores in the dataset were temporarily closed for refurbishment.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df1=pd.read_csv('/content/drive/MyDrive/Rossmann Stores Data (2).csv')
df2=pd.read_csv('/content/drive/MyDrive/store (1).csv')


<ipython-input-250-70e50507223c>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df1=pd.read_csv('/content/drive/MyDrive/Rossmann Stores Data (2).csv')


In [ ]:
df1.head(2)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1


In [ ]:
df2.head(2)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"


In [ ]:
df1.shape

(1017209, 9)

In [ ]:
df2.shape

(1115, 10)

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1112 non-null   float64
 4   CompetitionOpenSinceMonth  761 non-null    float64
 5   CompetitionOpenSinceYear   761 non-null    float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            571 non-null    float64
 8   Promo2SinceYear            571 non-null    float64
 9   PromoInterval              571 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


**DATA PREPROCESSING**

In [ ]:
df1.isnull().sum()

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [ ]:
df2.isnull().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [ ]:
store=df2.copy()
store['CompetitionDistance']=store['CompetitionDistance'].fillna(0)
store['CompetitionOpenSinceMonth']=store['CompetitionOpenSinceMonth'].fillna(0)
store_mode=store['CompetitionOpenSinceYear'].mode().iloc[0]
store['CompetitionOpenSinceYear']=store['CompetitionOpenSinceYear'].fillna(store_mode)
store['Promo2SinceWeek']=store['Promo2SinceWeek'].fillna(0)
year_mode=store['Promo2SinceYear'].mode().iloc[0]
store['Promo2SinceYear']=store['Promo2SinceYear'].fillna(year_mode)
interval_mode=store['PromoInterval'].mode().iloc()
store['PromoInterval']=store['PromoInterval'].fillna(interval_mode)

maping everyone to numerics

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
store['Storetype'] = label_encoder.fit_transform(store['StoreType'])
store['Assortment']=label_encoder.fit_transform(store['Assortment'])
store['PromoInterval']= store['PromoInterval'].map({'Jan,Apr,Jul,Oct':0, 'Feb,May,Aug,Nov':1,'Mar,Jun,Sept,Dec':2})

In [ ]:
store['PromoInterval'].value_counts()

0.0    335
1.0    130
2.0    106
Name: PromoInterval, dtype: int64

In [ ]:
store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   int64  
 3   CompetitionDistance        1115 non-null   float64
 4   CompetitionOpenSinceMonth  1115 non-null   float64
 5   CompetitionOpenSinceYear   1115 non-null   float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            1115 non-null   float64
 8   Promo2SinceYear            1115 non-null   float64
 9   PromoInterval              571 non-null    float64
 10  Storetype                  1115 non-null   int64  
dtypes: float64(6), int64(4), object(1)
memory usage: 95.9+ KB


In [ ]:
final=pd.merge(df1,store,on=['Store'],how='left')

In [ ]:
final.shape

(1017209, 19)

In [ ]:
final.describe()

,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Storetype
count,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,509178.000000,1.017209e+06
mean,5.584297e+02,3.998341e+00,5.773819e+03,6.331459e+02,8.301067e-01,3.815145e-01,1.786467e-01,9.351412e-01,5.415982e+03,4.926878e+00,2.010060e+03,5.005638e-01,1.164767e+01,2.011377e+03,0.615730,1.207467e+00
std,3.219087e+02,1.997391e+00,3.849926e+03,4.644117e+02,3.755392e-01,4.857586e-01,3.830564e-01,9.938011e-01,7.710253e+03,4.283543e+00,5.340757e+00,4.999999e-01,1.532393e+01,1.235230e+00,0.787033,1.365376e+00
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.900000e+03,0.000000e+00,0.000000e+00,2.009000e+03,0.000000,0.000000e+00
25%,2.800000e+02,2.000000e+00,3.727000e+03,4.050000e+02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e+02,0.000000e+00,2.008000e+03,0.000000e+00,0.000000e+00,2.011000e+03,0.000000,0.000000e+00
50%,5.580000e+02,4.000000e+00,5.744000e+03,6.090000e+02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.320000e+03,4.000000e+00,2.012000e+03,1.000000e+00,1.000000e+00,2.011000e+03,0.000000,0.000000e+00
75%,8.380000e+02,6.000000e+00,7.856000e+03,8.370000e+02,1.000000e+00,1.000000e+00,0.000000e+00,2.000000e+00,6.880000e+03,9.000000e+00,2.013000e+03,1.000000e+00,2.200000e+01,2.012000e+03,1.000000,3.000000e+00
max,1.115000e+03,7.000000e+00,4.155100e+04,7.388000e+03,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,7.586000e+04,1.200000e+01,2.015000e+03,1.000000e+00,5.000000e+01,2.015000e+03,2.000000,3.000000e+00


In [ ]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 19 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  int64  
 1   DayOfWeek                  1017209 non-null  int64  
 2   Date                       1017209 non-null  object 
 3   Sales                      1017209 non-null  int64  
 4   Customers                  1017209 non-null  int64  
 5   Open                       1017209 non-null  int64  
 6   Promo                      1017209 non-null  int64  
 7   StateHoliday               1017209 non-null  object 
 8   SchoolHoliday              1017209 non-null  int64  
 9   StoreType                  1017209 non-null  object 
 10  Assortment                 1017209 non-null  int64  
 11  CompetitionDistance        1017209 non-null  float64
 12  CompetitionOpenSinceMonth  1017209 non-null  float64
 13  CompetitionO

In [ ]:
final['Date']=pd.to_datetime(final['Date'])
final['salemonth']=final['Date'].dt.month
final['StateHoliday']=final['StateHoliday'].replace({0:'0'})
cols_drop = ['Date', 'Customers', 'Store']
final=final.drop(cols_drop,axis=1)


In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
final['storetype'] = label_encoder.fit_transform(final['StoreType'])
final['assortment']=label_encoder.fit_transform(final['Assortment'])
final['promoInterval']= final['PromoInterval'].map({'Jan,Apr,Jul,Oct':0, 'Feb,May,Aug,Nov':1,'Mar,Jun,Sept,Dec':2})

In [ ]:
final = pd.get_dummies(final, drop_first=True)

In [ ]:
final.sample(3)

,DayOfWeek,Sales,Open,Promo,SchoolHoliday,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,...,salemonth,storetype,assortment,promoInterval,StateHoliday_a,StateHoliday_b,StateHoliday_c,StoreType_b,StoreType_c,StoreType_d
731124,6,12304,1,0,0,2,50.0,0.0,2013.0,0,...,9,0,2,NaN,0,0,0,0,0,0
451304,5,10872,1,1,0,2,11260.0,0.0,2013.0,1,...,5,3,2,NaN,0,0,0,0,0,1
963249,1,7262,1,1,0,0,2100.0,8.0,2013.0,1,...,2,0,0,NaN,0,0,0,0,0,0


In [ ]:
final['storetype']=final['storetype'].astype(int)

In [ ]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 24 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   DayOfWeek                  1017209 non-null  int64  
 1   Sales                      1017209 non-null  int64  
 2   Open                       1017209 non-null  int64  
 3   Promo                      1017209 non-null  int64  
 4   SchoolHoliday              1017209 non-null  int64  
 5   Assortment                 1017209 non-null  int64  
 6   CompetitionDistance        1017209 non-null  float64
 7   CompetitionOpenSinceMonth  1017209 non-null  float64
 8   CompetitionOpenSinceYear   1017209 non-null  float64
 9   Promo2                     1017209 non-null  int64  
 10  Promo2SinceWeek            1017209 non-null  float64
 11  Promo2SinceYear            1017209 non-null  float64
 12  PromoInterval              509178 non-null   float64
 13  Storetype   

In [ ]:
final.shape

(1017209, 24)

In [ ]:
final=final.drop(columns=['StoreType_b'])

In [ ]:
final=final.drop(columns=['StoreType_c','StoreType_d'])

In [ ]:
final.shape

(1017209, 21)

In [ ]:
final

,DayOfWeek,Sales,Open,Promo,SchoolHoliday,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,...,Promo2SinceYear,PromoInterval,Storetype,salemonth,storetype,assortment,promoInterval,StateHoliday_a,StateHoliday_b,StateHoliday_c
0,5,5263,1,1,1,0,1270.0,9.0,2008.0,0,...,2011.0,NaN,2,7,2,0,NaN,0,0,0
1,5,6064,1,1,1,0,570.0,11.0,2007.0,1,...,2010.0,0.0,0,7,0,0,NaN,0,0,0
2,5,8314,1,1,1,0,14130.0,12.0,2006.0,1,...,2011.0,0.0,0,7,0,0,NaN,0,0,0
3,5,13995,1,1,1,2,620.0,9.0,2009.0,0,...,2011.0,NaN,2,7,2,2,NaN,0,0,0
4,5,4822,1,1,1,0,29910.0,4.0,2015.0,0,...,2011.0,NaN,0,7,0,0,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,2,0,0,0,1,0,1900.0,6.0,2014.0,1,...,2013.0,0.0,0,1,0,0,NaN,1,0,0
1017205,2,0,0,0,1,2,1880.0,4.0,2006.0,0,...,2011.0,NaN,2,1,2,2,NaN,1,0,0
1017206,2,0,0,0,1,2,9260.0,0.0,2013.0,0,...,2011.0,NaN,0,1,0,2,NaN,1,0,0
1017207,2,0,0,0,1,2,870.0,0.0,2013.0,0,...,2011.0,NaN,0,1,0,2,NaN,1,0,0


In [ ]:
final=final.drop(columns=['Storetype','PromoInterval'],axis=1)

In [ ]:
final.shape

(1017209, 19)

In [ ]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 19 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   DayOfWeek                  1017209 non-null  int64  
 1   Sales                      1017209 non-null  int64  
 2   Open                       1017209 non-null  int64  
 3   Promo                      1017209 non-null  int64  
 4   SchoolHoliday              1017209 non-null  int64  
 5   Assortment                 1017209 non-null  int64  
 6   CompetitionDistance        1017209 non-null  float64
 7   CompetitionOpenSinceMonth  1017209 non-null  float64
 8   CompetitionOpenSinceYear   1017209 non-null  float64
 9   Promo2                     1017209 non-null  int64  
 10  Promo2SinceWeek            1017209 non-null  float64
 11  Promo2SinceYear            1017209 non-null  float64
 12  salemonth                  1017209 non-null  int64  
 13  storetype   

In [ ]:
final=final.drop(['promoInterval','assortment'],axis=1)

In [ ]:
final.shape

(1017209, 17)

MODELING

In [ ]:
X=final.drop(['Sales'],axis=1)
y=final['Sales']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((813767, 16), (203442, 16), (813767,), (203442,))

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score as r2, mean_squared_error as mse
import math
from statistics import sqrt
dt_regressor=DecisionTreeRegressor(max_depth=5)
dt_regressor.fit(X_train, y_train)
y_pred=dt_regressor.predict(X_test)
y_train_pred=dt_regressor.predict(X_train)
print('dt_regressor R^2: ', r2(y_test,y_pred))
print('dt_regressor RMSE: ', math.sqrt(mse(y_test, y_pred)))


dt_regressor R^2:  0.5796454472107977
dt_regressor RMSE:  2502.5624536144182


In [ ]:
from sklearn.linear_model import LinearRegression
reg=LinearRegression()
reg.fit(X_train,y_train)
y_pred=reg.predict(X_test)
print('R^2: ', r2(y_test,y_pred))
print('RMSE: ',math.sqrt(mse(y_test,y_pred)))


R^2:  0.5523197161969445
RMSE:  2582.6230974016394


In [ ]:
from sklearn.ensemble import RandomForestRegressor
RandomForestRegressor(n_estimators=100,
                              random_state=3,
                              max_samples=0.5,
                              max_features=0.75,
                              max_depth=15)
rf_regressor=RandomForestRegressor()
rf_regressor.fit(X_train, y_train)
y_pred=rf_regressor.predict(X_test)
print('rf_regressor R^2: ', r2(y_test,y_pred))
print('rf_regressor RMSE: ', math.sqrt(mse(y_test, y_pred)))

rf_regressor R^2:  0.5826278822386088
rf_regressor RMSE:  2493.668751948458


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
rf_regresso=GradientBoostingRegressor(n_estimators=500)
rf_regresso=GradientBoostingRegressor()
rf_regresso.fit(X_train, y_train)
y_pred=rf_regresso.predict(X_test)
print('rf_regressor R^2: ', r2(y_test,y_pred))
print('rf_regressor RMSE: ', math.sqrt(mse(y_test, y_pred)))

rf_regressor R^2:  0.923127503864393
rf_regressor RMSE:  1070.1940654960304


In [ ]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
rf_regress=Lasso()
rf_regress=Lasso(alpha=0.0001)
rf_regress.fit(X_train, y_train)
y_pred=rf_regress.predict(X_test)
print('rf_regressor R^2: ', r2(y_test,y_pred))
print('rf_regressor RMSE: ', math.sqrt(mse(y_test, y_pred)))

rf_regressor R^2:  0.5523197161094849
rf_regressor RMSE:  2582.6230976539123


In [ ]:
from xgboost import XGBRegressor
rf_regress=XGBRegressor(n_estimators=45,max_depth=5,learning_rate=0.5)
rf_regress=XGBRegressor()
rf_regress.fit(X_train, y_train)
y_pred=rf_regress.predict(X_test)
print('rf_regressor R^2: ', r2(y_test,y_pred))
print('rf_regressor RMSE: ', math.sqrt(mse(y_test, y_pred)))


rf_regressor R^2:  0.8870710430207683
rf_regressor RMSE:  1297.1186545524542
